# 工具  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
工具是代理调用来执行操作的组件。它们通过允许模型通过定义良好的输入和输出来与世界交互，从而扩展模型的功能。

工具封装了一个可调用函数及其输入模式。这些可以传递给兼容的聊天模型，允许模型决定是否调用工具以及使用什么参数。在这些场景中，工具调用使模型能够生成符合指定输入模式的请求。

LangChain 支持多种工具格式和工具集。这里涵盖一些常见场景，更多信息请查看[文档](https://docs.langchain.com/oss/python/langchain/tools)。

## 初始化
加载并检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")

DASHSCOPE_API_KEY=****931f
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****ef8f
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****s-56
LANGSMITH_ENDPOINT=****.com


## 计算器示例

在这个示例中，LLM 会利用文档字符串以及推断出的参数和类型来判断何时、如何调用该工具。

创建工具最简单的方法是使用 @tool 装饰器。默认情况下，函数的文档字符串成为工具的描述，帮助模型理解何时使用它

类型提示是必需的，因为它们定义了工具的输入模式。文档字符串应具备信息性且简洁，帮助模型理解工具的目的。

In [4]:
from typing import Literal
from langchain.tools import tool

@tool
def real_number_calculator(
    a:float,b:float,operation:Literal["add","subtract","multiply","divide"]
)->float:
    """一个简单的计算器，支持加减乘除四则运算"""
    print("🧮 调用计算器工具")
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("除数不能为零")
        return a / b
    else:
        raise ValueError("不支持的操作类型")

In [5]:
from langchain.agents import create_agent
from langchain_qwq import ChatQwen
import os

#创建模型实例
model=ChatQwen(
    model="qwen3-max", 
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    )

# 创建Agent实例
agent=create_agent(
    model,
    tools=[real_number_calculator],
    system_prompt="你是一个乐于助人的助手，擅长使用计算器进行数学运算。",
    )

调用工具

In [6]:
result=agent.invoke({"messages":[
    {"role": "user", "content": "计算一下 15 除以 3，然后加上 7。"},
    ]})
messages=result["messages"]
for message in messages:
    message.pretty_print()

🧮 调用计算器工具
🧮 调用计算器工具
================================ Human Message =================================

计算一下 15 除以 3，然后加上 7。
================================== Ai Message ==================================
Tool Calls:
  real_number_calculator (call_ba05eb15ccee400a9c5a9339)
 Call ID: call_ba05eb15ccee400a9c5a9339
  Args:
    a: 15
    b: 3
    operation: divide
  real_number_calculator (call_ab4f7bf0326d4f5ca3dd99bf)
 Call ID: call_ab4f7bf0326d4f5ca3dd99bf
  Args:
    a: 5
    b: 7
    operation: add
================================= Tool Message =================================
Name: real_number_calculator

5.0
================================= Tool Message =================================
Name: real_number_calculator

12.0
================================== Ai Message ==================================

首先，15 除以 3 等于 5。然后，5 加上 7 等于 12。

最终结果是 **12**。


工具描述会产生很大影响。 

因为输入是整数，这次可能不会调用你的计算器工具。（每次运行结果可能不同）

## 添加更详细的描述
尽管基本描述通常已足够，LangChain 也支持更丰富的描述。下面的示例使用了一种方式：Google 风格的参数描述。配合 `parse_docstring=True` 使用时，会解析并将参数描述传递给模型。你可以重命名工具并修改其描述。当你共享一个标准工具但希望加入面向代理的指令时，这会很有效。

In [ ]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",       #默认情况下，工具名称来源于函数名称。需要更具体描述时可以优先覆盖
    parse_docstring=True,
    description=(
        "对两个实数执行基础算术运算。"
        "当你需要对任何数字进行运算时都可以使用，即便是整数。"
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """对两个实数执行基础算术运算。

    Args:
        a (float): 第一个操作数。
        b (float): 第二个操作数。
        operation (Literal['add','subtract','multiply','divide']): 要执行的操作。

    Returns:
        float: 计算结果。

    Raises:
        ValueError: 当操作无效或尝试被零除时抛出。
    """
    print("🧮  调用计算器工具")
    # 执行指定的运算
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("不允许被零除。")
        return a / b
    else:
        raise ValueError(f"无效的操作: {operation}")


In [8]:
result=agent.invoke({"messages":[
    {"role": "user", "content": "计算一下 15 除以 3，然后加上 7。"},
    ]})
messages=result["messages"]
for message in messages:
    message.pretty_print()

🧮 调用计算器工具🧮 调用计算器工具

================================ Human Message =================================

计算一下 15 除以 3，然后加上 7。
================================== Ai Message ==================================
Tool Calls:
  real_number_calculator (call_3b91f10b9eca412aa4668c75)
 Call ID: call_3b91f10b9eca412aa4668c75
  Args:
    a: 15
    b: 3
    operation: divide
  real_number_calculator (call_61c16c7159e646cfb39ac62e)
 Call ID: call_61c16c7159e646cfb39ac62e
  Args:
    a: 5
    b: 7
    operation: add
================================= Tool Message =================================
Name: real_number_calculator

5.0
================================= Tool Message =================================
Name: real_number_calculator

12.0
================================== Ai Message ==================================

计算结果是 12。


## 工具访问上下文

工具可以通过 `ToolRuntime` 参数访问运行时信息，该参数提供：

- 状态 State - 在执行过程中流动的可变数据（例如，消息、计数器、自定义字段）
- 上下文 Context - 不可变的配置，如用户 ID、会话详情或特定应用的配置
- 存储 Store - 跨对话的持久长期记忆
- 流式写入器 Stream Writer- 在工具执行时流式传输自定义更新
- 配置 Config - RunnableConfig 用于执行
- 工具调用 ID Tool Call ID - 当前工具调用的 ID

<img src="./assets/AccessingContext.png" align="left" width="700">